In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
import torch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [20]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
print(torch.cuda.current_device())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

True
1
NVIDIA GeForce RTX 3070 Ti Laptop GPU
0


In [4]:
# import all the required data
train_df = pd.read_csv("./data/train.csv")
train_df.head()

C:\Users\krish\AppData\Local\Temp\ipykernel_8064\3118064815.py:2: DtypeWarning: Columns (4,13,14,16,47) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("./data/train.csv")


,resultId,racerId,driverId,constructorId,number,grid,position_x,positionText_x,positionOrder,points,...,points_y,position,positionText_y,wins,constructorRef,company,nationality_y,url,status,result_driver_standing
0,20025,833,642,51,2,1,1,1,1,9.0,...,3.33,3,3,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,922731975
1,20025,833,642,51,2,1,1,1,1,9.0,...,6.33,3,3,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,923172525
2,20025,833,642,51,2,1,1,1,1,9.0,...,6.33,4,4,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,923833350
3,20025,833,642,51,2,1,1,1,1,9.0,...,10.33,3,3,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,925195050
4,20025,833,642,51,2,1,1,1,1,9.0,...,10.33,4,4,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,926576775


In [5]:
test_df = pd.read_csv("./data/test.csv")
test_df.head()

C:\Users\krish\AppData\Local\Temp\ipykernel_8064\2707460454.py:1: DtypeWarning: Columns (13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv("./data/test.csv")


,resultId,racerId,driverId,constructorId,number,grid,position_x,positionText_x,positionOrder,points,...,driverStandingsId,raceId_y,points_y,wins,constructorRef,company,nationality_y,url,status,result_driver_standing
0,23949,997,4,1,14,13,8,8,8,4.0,...,12474,93,16.0,0,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,+1 Lap,298739826
1,23949,997,4,1,14,13,8,8,8,4.0,...,12494,94,21.0,0,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,+1 Lap,299218806
2,23949,997,4,1,14,13,8,8,8,4.0,...,12514,95,21.0,0,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,+1 Lap,299697786
3,23949,997,4,1,14,13,8,8,8,4.0,...,12534,96,25.0,0,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,+1 Lap,300176766
4,23949,997,4,1,14,13,8,8,8,4.0,...,12554,97,25.0,0,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,+1 Lap,300655746


In [6]:
validation = pd.read_csv("./data/validation.csv")
validation.head()

C:\Users\krish\AppData\Local\Temp\ipykernel_8064\3796945544.py:1: DtypeWarning: Columns (13,16,36) have mixed types. Specify dtype option on import or set low_memory=False.
  validation = pd.read_csv("./data/validation.csv")


,resultId,racerId,driverId,constructorId,number,grid,position_x,positionText_x,positionOrder,points,...,points_y,position,positionText_y,wins,constructorRef,company,nationality_y,url,status,result_driver_standing
0,22109,899,17,9,2,4,2,2,2,18.0,...,28.0,8,8,0,red_bull,Red Bull,Austrian,http://en.wikipedia.org/wiki/Red_Bull_Racing,Finished,1410907944
1,22109,899,17,9,2,4,2,2,2,18.0,...,53.0,4,4,1,red_bull,Red Bull,Austrian,http://en.wikipedia.org/wiki/Red_Bull_Racing,Finished,1411438560
2,22109,899,17,9,2,4,2,2,2,18.0,...,78.0,1,1,2,red_bull,Red Bull,Austrian,http://en.wikipedia.org/wiki/Red_Bull_Racing,Finished,1412499792
3,22109,899,17,9,2,4,2,2,2,18.0,...,93.0,1,1,2,red_bull,Red Bull,Austrian,http://en.wikipedia.org/wiki/Red_Bull_Racing,Finished,1413030408
4,22109,899,17,9,2,4,2,2,2,18.0,...,103.0,3,3,2,red_bull,Red Bull,Austrian,http://en.wikipedia.org/wiki/Red_Bull_Racing,Finished,1413561024


In [7]:
# understanding data
train_df.columns

Index(['resultId', 'racerId', 'driverId', 'constructorId', 'number', 'grid',
       'position_x', 'positionText_x', 'positionOrder', 'points', 'laps',
       'time_x', 'timetaken_in_millisec', 'fastestLap', 'rank',
       'fastestLapTime', 'max_speed', 'statusId', 'year', 'round', 'circuitId',
       'grand_prix', 'date', 'time_y', 'url_x', 'fp1_date', 'fp1_time',
       'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date',
       'quali_time', 'sprint_date', 'sprint_time', 'driverRef', 'driver_num',
       'driver_code', 'forename', 'surname', 'dob', 'nationality', 'url_y',
       'driverStandingsId', 'raceId_y', 'points_y', 'position',
       'positionText_y', 'wins', 'constructorRef', 'company', 'nationality_y',
       'url', 'status', 'result_driver_standing'],
      dtype='object')

In [8]:
# dropping columns that are useless
new_train_df = train_df[
    ["result_driver_standing", "points_y", "company", "round",
     "status", "wins", "grand_prix", "year", "points", "positionOrder",
     "resultId", "racerId", "position"]]
new_train_df.head()

,result_driver_standing,points_y,company,round,status,wins,grand_prix,year,points,positionOrder,resultId,racerId,position
0,922731975,3.33,Alfa Romeo,1,Finished,0,British Grand Prix,1950,9.0,1,20025,833,3
1,923172525,6.33,Alfa Romeo,1,Finished,0,British Grand Prix,1950,9.0,1,20025,833,3
2,923833350,6.33,Alfa Romeo,1,Finished,0,British Grand Prix,1950,9.0,1,20025,833,4
3,925195050,10.33,Alfa Romeo,1,Finished,0,British Grand Prix,1950,9.0,1,20025,833,3
4,926576775,10.33,Alfa Romeo,1,Finished,0,British Grand Prix,1950,9.0,1,20025,833,4


In [9]:
new_test_df = test_df[
    ["result_driver_standing", "points_y", "company", "round",
     "status", "wins", "grand_prix", "year", "points", "positionOrder",
     "resultId", "racerId"]]
new_test_df.head()

,result_driver_standing,points_y,company,round,status,wins,grand_prix,year,points,positionOrder,resultId,racerId
0,298739826,16.0,McLaren,9,+1 Lap,0,Austrian Grand Prix,2018,4.0,8,23949,997
1,299218806,21.0,McLaren,9,+1 Lap,0,Austrian Grand Prix,2018,4.0,8,23949,997
2,299697786,21.0,McLaren,9,+1 Lap,0,Austrian Grand Prix,2018,4.0,8,23949,997
3,300176766,25.0,McLaren,9,+1 Lap,0,Austrian Grand Prix,2018,4.0,8,23949,997
4,300655746,25.0,McLaren,9,+1 Lap,0,Austrian Grand Prix,2018,4.0,8,23949,997


In [10]:
finished_train = new_train_df[new_train_df["status"] == "Finished"]
finished_train = finished_train.drop(["status"], axis=1)

In [11]:
finished_test = new_train_df[new_train_df["status"] == "Finished"]
finished_test = finished_test.drop(["status"], axis=1)

In [12]:
# encoding all columns
label_encoders = {}
for column in ['company', 'grand_prix']:
    le = LabelEncoder()
    finished_train[column] = le.fit_transform(finished_train[column])
    label_encoders[column] = le
    
label_encoders = {}
for column in ['company', 'grand_prix']:
    le = LabelEncoder()
    finished_test[column] = le.fit_transform(finished_test[column])
    label_encoders[column] = le


In [13]:
X = finished_train.drop(columns=['position'])
y = finished_train['position']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1).to(device)

In [23]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [24]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model, loss function and optimizer
model = Net().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [25]:
# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/50], Loss: 14.1904
Epoch [2/50], Loss: 16.0543
Epoch [3/50], Loss: 16.8759
Epoch [4/50], Loss: 8.1276
Epoch [5/50], Loss: 61.9998
Epoch [6/50], Loss: 12.1174
Epoch [7/50], Loss: 0.4932
Epoch [8/50], Loss: 3.5576
Epoch [9/50], Loss: 13.4306


KeyboardInterrupt: 

In [ ]:
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)

# Print sample predictions
print(f'Predictions: {predictions[:5].cpu().numpy()}')